**Author**: Brandon Le  
**Workshop**: Reproducible research using Jupyter Notebook: RNA-seq data analysis workflow  
**Git Repo**: https://github.com/bioinformatics-workshop/RNA-seq-workshop-Jupyter-notebook

---

Below is a general workflow for the processing and analysis of an RNA-seq dataset. 
Code chunks below were implemented on the UCR HPCC cluster.

# Data Preparation

## Obtain SRA sample list
For this demo, we will use a publicly available dataset from the [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra) (SRA) hosted at the National Center for Biotechnology Information (NCBI). To download the demo dataset, we will obtain a SRA sample list containing information for a BioProject stored in the SRA database using the BioProject ID: PRJNA950346.

We will first use the following shell script to obtain information about the BioProject.

In [ ]:
# Do not execute
sbatch code/sra-info_download.sh

The script will generate a `PRJNA950346.metadata.tmp` file in the `raw` folder. This file contain relevant information about the dataset including accession numbers for each sample. Lets examine the content of the file.

In [ ]:
cat -n raw/PRJNA950346.metadata.tmp

## Data Download

We will use the accession information (SRRXXXXXXXX) to download the data from the NCBI SRA database.

In [ ]:
# Do not execute
# sbatch -a 1-6 code/sra_download.sh

The shell script is executed as an array. The `-a 1-6` portion of the command allows processing the array lines 1 to 6 corresponding to samples 1 to 6 from the `PRJNA950346.metadata.tmp` file. This means that all six samples will be processed in parallel rather than sequentially.

**For Example:**  

- To process the first two samples, we would run `-a 1-2`.   
- To process sample rows 3, 5, and 6, we run `-a 3,5,6`. This allows us to submit individual jobs for each sample in parallel.

## Generate metadata file
A metadata file is used to describe the sequencing project and used for downstream analysis.

**NOTE:** Metadata files are great for storing relevant information about the sample and project.

For this demo, we created a file containing seven columns (the **first three columns are required** and the *remaining columns are optional*)

The metadata file consists of seven columns:  
samplename, fq1, fq2, srr_id, ecotype, genotype, treatment, tissue, biorep
  
-   **samplename** : sample name for labeling (required)
-   **fq1** : filepath to read1 (required)
-   **fq2** : filepath to read2 (required if paired-end)
-   **srr_id** : SRR ID from SRA run
-   **ecotype**: Col-0
-   **genotype**: WT or mir163 mutant
-   **treatment**: 7 day old plants
-   **tissue**: seedlings
-   **biorep**: biological replicate number

The `samplename` is used for all downstream analysis.

In [ ]:
# Do not execute
# Rscript --vanilla code/metadata.R

In [ ]:
cat metadata/metadata.csv

## Genome Download

Genome sequence and annotation files were downloaded for the Arabidopsis genome. 

Resources available at: [Araport 11](https://www.arabidopsis.org/download/index-auto.jsp?dir=%2Fdownload_files%2FGenes%2FAraport11_genome_release)

In [ ]:
# Do not execute
# sbatch code/download-genome.sh

# RNA-Seq Data Processing

The RNA-seq data processing will consist of three major steps:

1. Generate the index for the reference genome
2. QC and alignment
3. Differential expression analysis

## Indexing the Reference genome

Indexing provides a fast method to align the reads to the reference genome. We will use the [STAR](https://academic.oup.com/bioinformatics/article/29/1/15/272537) aligner to first index the genome and then alignment.

In [ ]:
sbatch code/00-rnaseq-pre-process.sh

## QC and read alignments

We will run quality control on the raw reads using `Fastqc` and `trim_galore` to remove low-quality bases and adapters. Then align the trimmed reads to the reference genome and generate outputs for visualization on a genome browser.

In [ ]:
sbatch code/01-rnaseq-align.sh

## Differential Expression analysis

After QC and alignment, we can quantify the read counts per gene and run differential expression analysis using [DESeq2](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8). The differential expression analysis pipeline will generate PCA plots, DEG lists, and volcano plots.

In [ ]:
sbatch code/02-rnaseq-differential-abundance.sh